[本部分的官方代码地址](https://colab.research.google.com/drive/1V0CutSIfmtgDJg1XIkEnGteJuw0u7qT-#scrollTo=h1KXYz-Nauwn)：https://colab.research.google.com/drive/1V0CutSIfmtgDJg1XIkEnGteJuw0u7qT-#scrollTo=h1KXYz-Nauwn


这一部分主要介绍的是如何使用vxm库里的方法对图像进行变换，代码中的方法是随机生成一个矩阵，然后根据该矩阵对图像进行仿射变换。


**环境要求：tensorflow2.4,VoxelMorph**

## 1. Import Module 库的导入

In [ ]:
!pip install voxelmorph
import voxelmorph as vxm
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## 2. Input and pre-process of the Image 图像的输入与预处理：

教程中使用的是mnist的数据集，数据集的预处理步骤包括:

- 对原始图像进行边缘填充
- 灰度归一化（具体作用参考：https://blog.csdn.net/qq_41383956/article/details/88593538） 

In [ ]:
# 加载mnist数据集
# 其标准输出应为： (x_train, y_train), (x_test, y_test)，但是只需要x_train数据展示，所以其他的丢掉
(x_train,_),_ = tf.keras.datasets.mnist.load_data()
# 灰度归一化，从0-255压缩到0-1
im = x_train[0,...].astype('float')/255
# 边缘填充
# 这一步的目的是，在后面对图像进行变换时，原本的Mnist数据集的28*28在变换后，
# 数字可能会移出图像区域，所以扩大原始数据的大小，也就是空白部分，方便展示变换的效果。
# pad_amt设置为10，及补充的区域为10个pixel
pad_amt = 10
# np.pad(需要填充的array，((上，下),(左，右))，mode=constant...),这一步是为了增加边缘，可以理解为padding
# 原始数据28*28，填补大小，上下左右各10，处理后数据48*48
im = np.pad(im,((pad_amt,pad_amt),(pad_amt,pad_amt)))

![bofore](./Image/5-before-pre-process.png)
![after](./Image/5-after-pre-process.png)

## 3. Create transformation matrix manually 手动创建变换矩阵：

In [ ]:
# 手动生成仿射变换矩阵，方便后面affine操作

# 创建主对角矩阵
aff = np.eye(3) 
 # 在左上半部分的2*2区域加入随机噪声
aff[:2,:2]+=np.random.randn(2,2)*0.1
# 前两行的第三列的内容使用(-10,10)之间的均匀随机采样数字来替换
#  np.random.uniform(low,high,size)，使用(2,)的原因是aff[:2,2]数组就是一个两行一列的值
aff[:2, 2] = np.random.uniform(-10, 10, (2, ))
# 对上面计算后的矩阵求逆
aff_inv = np.linalg.inv(aff)

# 手动生成annotation变换矩阵，方便后面warp操作
margin=10
nb_annotations = 5
 # 创建一个列表，其中包含两个annotations，每个中包含nb_annotations个随机数字，范围在(margin,f-margin)之间
annotations = [np.random.uniform(margin,f-margin,nb_annotations) for f in im.shape]
# np.stack的简单用法在我的notion中有说明：
# https://sandy-property-d5e.notion.site/np-stack-48a69e31be084aa98cd15ce7d093c2ec
annotations = np.stack(annotations,1)

处理后的数据分别为：
aff_inv:
$$ \begin{bmatrix}
  1.01& -3.137 & -7.602\\
  5.857&  8.561& 3.138\\
  0& 0 &1
\end{bmatrix}$$
annotations:
$$
\begin{bmatrix}
 23.46& -3.3\\
  2.34&7.33 \\
  6.42& 14.34\\
  5.32& 37.34\\
  7.14&17.34
\end{bmatrix}
$$

## 4. Warp Data

In [ ]:
# np.newaxis 的功能是增加新的维度。放在第一个，给行上增加维度，放在最后一个，给列上增加维度
im_keras = im[np.newaxis,...,np.newaxis] # (1, 48, 48, 1)
aff_keras = aff[np.newaxis,:2,:] #(1, 2, 3)
annotations_keras = annotations[np.newaxis,...] # (1,5,2)

# warp image
# 调用vxm库里的SpatialTransformer类，([im_keras, aff_keras])放在后面是什么用法暂时还没搞懂
# ([im_keras, aff_keras])分别代表的是图像数据和形变数据，通过空间变换将形变数据作用到图像数据中
im_warped = vxm.layers.SpatialTransformer()([im_keras, aff_keras])
im_warped = im_warped[0, ..., 0] # 取im_warped中的第0行第0列

# 获取取逆操作后的仿射矩阵的密集场Dense field
# 此处的affine_to_dense_shift和官方的教程不同，因为新版的vxm已经更新为此方法，此处已通过Issue询问过开发者
# vxm.utils.affine_to_dense_shift(array,shape,shift_center=True)
# 最后[np.newaxis, ...]的作用等价于field_inv = field_inv[np.newaxis, ...]，即给输出的结果的第一个位置增加一个维度
field_inv = vxm.utils.affine_to_dense_shift(aff_inv[:-1,:], im.shape, shift_center=True)[np.newaxis, ...]

# warp annotations
# 我的理解是：annotation是一些随机生成的点，在变换前后的图像中都是存在的
# 其作用是，帮助更明显的看出图像变化的方向和形式（涉及形变、整体移动的方向等信息）
# data为长度为2的列表，存储的分别是annotations_keras, field_inv，且两个都被转换为tf.Tensor形式，用于输入到vxm.utils.point_spatial_transformer中
data = [tf.convert_to_tensor(f, dtype=tf.float32) for f in [annotations_keras, field_inv]]
# 将辅助点和形变场都放入 vxm.utils.point_spatial_transformer，获取辅助点在该形变场下的变换信息
# [0,...]：从[1,5,2]中获取第0维度的信息=>[5,2]
annotations_warped = vxm.utils.point_spatial_transformer(data)[0, ...].numpy()

## 5.Show result 展示结果

In [ ]:
plt.figure()
# 分别展示初始的图像和生成的辅助点
plt.subplot(1, 2, 1)
plt.imshow(im, cmap='gray')
plt.plot(*[annotations[:, f] for f in [1, 0]], 'o')  
plt.axis('off')

# 分别展示变换后的图像和变换后的辅助点
plt.subplot(1, 2, 2)
plt.imshow(im_warped, cmap='gray')
plt.plot(*[annotations_warped[:, f] for f in [1, 0]], 'o')
plt.axis('off');

![result](./Image/result-of-affine.png)